In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.ts.wep.utils import forwardModelPair
from lsst.ts.wep import Instrument

In [ ]:
# Make pixels smaller to create smoother images
inst = Instrument(pixelSize=0.5e-5)

In [ ]:
def map_circles(A, m, drho, dtheta, N = 5, ax=None, center=True, **kwargs):
    # Enforce plotting defaults if not specified in kwargs
    kwargs = {"c": "C1", "lw": 0.5, "zorder": 10} | kwargs
        
    # Create figure
    if ax is None:
        fig, ax = plt.subplots()

    for r in np.linspace(0.61, 1, N):
        # Circle
        r0 = r * np.ones(10_000)
        t0 = np.linspace(0, 2 * np.pi, 10_000)
        x0 = r0 * np.cos(t0)
        y0 = r0 * np.sin(t0)

        # Perturb
        r1 = r0 + A * drho(r0, t0, m)
        t1 = t0 + A * dtheta(r0, t0, m)
        x1 = r1 * np.cos(t1)
        y1 = r1 * np.sin(t1)

        # Plot
        ax.plot(x1, y1, **kwargs)

    # Center the donut
    if center:
        x0 = np.mean(ax.get_xlim())
        y0 = np.mean(ax.get_ylim())
    else:
        x0, y0 = 0, 0
    L = 1.25
    ax.set(xlim=(x0-L, x0+L), ylim=(y0-L, y0+L), aspect="equal", xticks=[], yticks=[])

def map_circles_xy(A, m, dx, dy, N = 5, ax=None, center=True, **kwargs):
    # Enforce plotting defaults if not specified in kwargs
    kwargs = {"c": "C1", "lw": 0.5, "zorder": 10} | kwargs
    
    # Create figure
    if ax is None:
        fig, ax = plt.subplots()

    # Create theta grid
    t0 = np.linspace(0, 2 * np.pi, 10_000)

    for r in np.linspace(0.61, 1, N):
        # Circle
        r0 = r * np.ones(10_000)
        x0 = r0 * np.cos(t0)
        y0 = r0 * np.sin(t0)

        # Perturb
        x1 = x0 + A * dx(r0, t0, m)
        y1 = y0 + A * dy(r0, t0, m)

        # Plot
        ax.plot(x1, y1, **kwargs)

    # Center the donut
    if center:
        x0 = np.mean(ax.get_xlim())
        y0 = np.mean(ax.get_ylim())
    else:
        x0, y0 = 0, 0
    L = 1.25
    ax.set(xlim=(x0-L, x0+L), ylim=(y0-L, y0+L), aspect="equal", xticks=[], yticks=[])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(3.5, 3.8), constrained_layout=True, dpi=150)

axes[0, 0].set_title("Defocus\n$\\nu = 1,\, m=0$")
axes[0, 1].set_title("Tilt\n$\\nu = 1,\, m=1$")

# Defocus
drho = lambda rho, theta, m: -rho
dtheta = lambda rho, theta, m: 0
map_circles(0.25, 0, drho, dtheta, ax=axes[0, 0])

# Tilt
drho = lambda rho, theta, m: -np.sin(theta)
dtheta = lambda rho, theta, m: -np.cos(theta)
map_circles(0.2, 0, drho, dtheta, ax=axes[0, 1], center=False)

# Plot simulated donuts
for ax in axes[1]:
    ax.set(xticks=[], yticks=[])

sim_settings = {"seeing": 0.6, "fieldAngleIntra": (0, 0), "skyLevel": 0, "nPix": 400, "flat": False, "instConfig": inst}
zk = np.zeros(19)
zk[4 - 4] = 5e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 0].imshow(intra.image[20:-20, 20:-20], origin="lower")

zk = np.zeros(19)
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 1].imshow(intra.image[60:, 30:-30], origin="lower")

fig.savefig("figures/abberations_defocus_tilt.pdf")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(7, 3.8), constrained_layout=True, dpi=120)

axes[0, 0].set_title("Astigmatism\n$\\nu = 1,\, m=2$")
axes[0, 1].set_title("Trefoil\n$\\nu = 1,\, m=3$")
axes[0, 2].set_title("Quadrafoil\n$\\nu = 1,\, m=4$")
axes[0, 3].set_title("Pentafoil\n$\\nu = 1,\, m=5$")

# Primary astig and m-foils
drho = lambda rho, theta, m: -m * rho**(m - 1) * np.sin(m * theta)
dtheta = lambda rho, theta, m: -m * rho**(m - 1) * np.cos(m * theta)

map_circles(0.070, 2, drho, dtheta, ax=axes[0, 0])
map_circles(0.020, 3, drho, dtheta, ax=axes[0, 1])
map_circles(0.010, 4, drho, dtheta, ax=axes[0, 2])
map_circles(0.007, 5, drho, dtheta, ax=axes[0, 3])

# Plot simulated donuts
for ax in axes[1]:
    ax.set(xticks=[], yticks=[])

sim_settings = {"seeing": 0.6, "fieldAngleIntra": (0, 0), "skyLevel": 0, "nPix": 330, "instConfig": inst}
zk = np.zeros(19)
zk[5 - 4] = 7e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 0].imshow(intra.image, origin="lower")

zk = np.zeros(19)
zk[9 - 4] = 2e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 1].imshow(intra.image, origin="lower")

zk = np.zeros(19)
zk[15 - 4] = 1e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 2].imshow(intra.image, origin="lower")

zk = np.zeros(19)
zk[21 - 4] = 0.5e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 3].imshow(intra.image, origin="lower")

fig.savefig("figures/abberations_astig_mfoil.pdf")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(3.5, 3.8), constrained_layout=True, dpi=150)

axes[0, 0].set_title("Spherical\n$\\nu = 2,\, m=0$")
axes[0, 1].set_title("Coma\n$\\nu = 2,\, m=1$")

# Spherical
drho = lambda rho, theta, m: -2 * rho**3 + (1 + 0.61**2) * rho
dtheta = lambda rho, theta, m: 0
map_circles(0.17, 0, drho, dtheta, ax=axes[0, 0])

# Coma
dx = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * -np.sin(2 * theta)
dy = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * np.cos(2 * theta) - 6 * (1 + 0.61**2) * rho**2
map_circles_xy(0.03, 0, dx, dy, ax=axes[0, 1])

# Plot simulated donuts
for ax in axes[1]:
    ax.set(xticks=[], yticks=[])

sim_settings = {"seeing": 0.6, "fieldAngleIntra": (0, 0), "skyLevel": 0, "nPix": 400, "flat": False, "instConfig": inst}
zk = np.zeros(19)
zk[11 - 4] = 0.2e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 0].imshow(intra.image[20:-20, 20:-20], origin="lower")

zk = np.zeros(19)
zk[7 - 4] = 2e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 1].imshow(intra.image[:-60, 30:-30], origin="lower")

fig.savefig("figures/abberations_spherical_coma.pdf")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(7, 3.8), constrained_layout=True, dpi=120)

axes[0, 0].set_title("2nd Astigmatism\n$\\nu = 2,\, m=2$")
axes[0, 1].set_title("2nd Trefoil\n$\\nu = 2,\, m=3$")
axes[0, 2].set_title("2nd Quadrafoil\n$\\nu = 2,\, m=4$")
axes[0, 3].set_title("2nd Pentafoil\n$\\nu = 2,\, m=5$")

# Secondary astig and m-foils
drho = lambda rho, theta, m: -2 * m * rho**(2*m - 1) * 2 * np.sin(m * theta) + (m + 1) * rho**(m-1) * np.sin(m * theta)
dtheta = lambda rho, theta, m: -2 * m * rho**(2*m - 1) * np.cos(m * theta) + (m + 1) * rho**(m-1) * np.cos(m * theta)

map_circles(0.07/2.4, 2, drho, dtheta, ax=axes[0, 0])
map_circles(0.02/2.5, 3, drho, dtheta, ax=axes[0, 1])
map_circles(0.02/4, 4, drho, dtheta, ax=axes[0, 2])
map_circles(0.007/3, 5, drho, dtheta, ax=axes[0, 3])

# Plot simulated donuts
for ax in axes[1]:
    ax.set(xticks=[], yticks=[])

sim_settings = {"seeing": 0.6, "fieldAngleIntra": (0, 0), "skyLevel": 0, "nPix": 330, "flat": False, "instConfig": inst}
zk = np.zeros(19)
zk[13 - 4] = 0.75e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 0].imshow(intra.image, origin="lower")

zk = np.zeros(19)
zk[19 - 4] = 0.4e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 1].imshow(intra.image, origin="lower")

zk = np.zeros(30)
zk[25 - 4] = 0.22e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 2].imshow(intra.image, origin="lower")

zk = np.zeros(35)
zk[33 - 4] = 0.12e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 3].imshow(intra.image, origin="lower")

fig.savefig("figures/abberations_2nd_astig_mfoil.pdf")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(3.5, 3.8), constrained_layout=True, dpi=150)

axes[0, 0].set_title("2nd Spherical\n$\\nu = 2,\, m=0$")
axes[0, 1].set_title("2nd Coma\n$\\nu = 2,\, m=1$")

# 2nd Spherical
drho = lambda rho, theta, m: -5 * rho**5 + 5 * (1 + 0.61**2) * rho**3 - (1 + 3*0.61**2 + 0.61**4) * rho
dtheta = lambda rho, theta, m: 0
map_circles(0.17, 0, drho, dtheta, ax=axes[0, 0])

# 2nd Coma
A = 10 * (1 + 4 * 0.61**2 + 0.61**4)
B = 12 * (1 + 4 * 0.61**2 + 4 * 0.61**4 + 0.61**6)
C = 3 * (1 + 4 * 0.61**2 + 10 * 0.61**4 + 4 * 0.61**6 + 0.61**8)
drho = lambda rho, theta, m: -(5 * A * rho**4 - 3 * B * rho**2 + C) * np.sin(theta)
dtheta = lambda rho, theta, m: -(A * rho**4 - B * rho**2 + C) * np.cos(theta)
map_circles(0.003, 0, drho, dtheta, ax=axes[0, 1])

# Plot simulated donuts
for ax in axes[1]:
    ax.set(xticks=[], yticks=[])

sim_settings = {"seeing": 0.6, "fieldAngleIntra": (0, 0), "skyLevel": 0, "nPix": 400, "flat": False, "instConfig": inst}
zk = np.zeros(19)
zk[22 - 4] = 0.05e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 0].imshow(intra.image[20:-20, 20:-20], origin="lower")

zk = np.zeros(19)
zk[17 - 4] = 0.2e-6
_, intra, extra = forwardModelPair(zkCoeff=zk, **sim_settings)
axes[1, 1].imshow(intra.image[15:-25, 20:-20], origin="lower")

fig.savefig("figures/abberations_2nd_spherical_coma.pdf")

# Old plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3.5, 2), constrained_layout=True, dpi=150)

# Defocus
drho = lambda rho, theta, m: -rho
dtheta = lambda rho, theta, m: 0
map_circles(0.25, 0, drho, dtheta, ax=ax1)

# Tilt
drho = lambda rho, theta, m: -np.sin(theta)
dtheta = lambda rho, theta, m: -np.cos(theta)
map_circles(0.2, 0, drho, dtheta, ax=ax2, center=False)

ax1.set_title("Defocus\n$\\nu = 1,\, m=0$")
ax2.set_title("Tilt\n$\\nu = 1,\, m=1$")


plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3.5, 2), constrained_layout=True, dpi=150)

ax1.set_title("Spherical\n$\\nu = 2,\, m=0$")
ax2.set_title("Coma\n$\\nu = 2,\, m=1$")

# Spherical
drho = lambda rho, theta, m: -2 * rho**3 + (1 + 0.61**2) * rho
dtheta = lambda rho, theta, m: 0
map_circles(0.17, 0, drho, dtheta, ax=ax1)

# Coma
dx = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * -np.sin(2 * theta)
dy = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * np.cos(2 * theta) - 6 * (1 + 0.61**2) * rho**2
map_circles_xy(0.03, 0, dx, dy, ax=ax2)

plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(7, 2), constrained_layout=True, dpi=150)

drho = lambda rho, theta, m: -m * rho**(m - 1) * np.sin(m * theta)
dtheta = lambda rho, theta, m: -m * rho**(m - 1) * np.cos(m * theta)

map_circles(0.07, 2, drho, dtheta, ax=ax1)
map_circles(0.02, 3, drho, dtheta, ax=ax2)
map_circles(0.01, 4, drho, dtheta, ax=ax3)
map_circles(0.007, 5, drho, dtheta, ax=ax4)

ax1.set_title("Astigmatism\n$\\nu = 1,\, m=2$")
ax2.set_title("Trefoil\n$\\nu = 1,\, m=3$")
ax3.set_title("Quadrafoil\n$\\nu = 1,\, m=4$")
ax4.set_title("Pentafoil\n$\\nu = 1,\, m=5$")
plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(7, 2), constrained_layout=True, dpi=150)

ax1.set_title("2nd Astigmatism\n$\\nu = 2,\, m=2$")
ax2.set_title("2nd Trefoil\n$\\nu = 2,\, m=3$")
ax3.set_title("2nd Quadrafoil\n$\\nu = 2,\, m=4$")
ax4.set_title("2nd Pentafoil\n$\\nu = 2,\, m=5$")

# Primary astig and m-foils
drho = lambda rho, theta, m: -m * rho**(m - 1) * np.sin(m * theta)
dtheta = lambda rho, theta, m: -m * rho**(m - 1) * np.cos(m * theta)

map_circles(0.08, 2, drho, dtheta, ax=ax1, ls="-", c="k", zorder=0)
map_circles(0.02, 3, drho, dtheta, ax=ax2, ls="-", c="k", zorder=0)
map_circles(0.015, 4, drho, dtheta, ax=ax3, ls="-", c="k", zorder=0)
map_circles(0.007, 5, drho, dtheta, ax=ax4, ls="-", c="k", zorder=0)

# Secondary astig and m-foils
drho = lambda rho, theta, m: -2 * m * rho**(2*m - 1) * 2 * np.sin(m * theta) + (m + 1) * rho**(m-1) * np.sin(m * theta)
dtheta = lambda rho, theta, m: -2 * m * rho**(2*m - 1) * np.cos(m * theta) + (m + 1) * rho**(m-1) * np.cos(m * theta)

map_circles(0.07/2.4, 2, drho, dtheta, ax=ax1, ls="--", lw=0.85)
map_circles(0.02/2.5, 3, drho, dtheta, ax=ax2, ls="--", lw=0.85)
map_circles(0.02/4, 4, drho, dtheta, ax=ax3, ls="--", lw=0.85)
map_circles(0.007/3, 5, drho, dtheta, ax=ax4, ls="--", lw=0.85)

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3.5, 2), constrained_layout=True, dpi=150)

ax1.set_title("2nd Spherical\n$\\nu = 2,\, m=0$")
ax2.set_title("2nd Coma\n$\\nu = 2,\, m=1$")

# Spherical
drho = lambda rho, theta, m: -2 * rho**3 + (1 + 0.61**2) * rho
dtheta = lambda rho, theta, m: 0
map_circles(0.13, 0, drho, dtheta, ax=ax1, ls="-", c="k", zorder=0)

# Coma
dx = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * -np.sin(2 * theta)
dy = lambda rho, theta, m: 3 * (1 + 0.61**2) * rho**2 * np.cos(2 * theta) - 6 * (1 + 0.61**2) * rho**2 + 9
map_circles_xy(0.03, 0, dx, dy, ax=ax2, ls="-", c="k", zorder=0)


# 2nd Spherical
drho = lambda rho, theta, m: -5 * rho**5 + 5 * (1 + 0.61**2) * rho**3 - (1 + 3*0.61**2 + 0.61**4) * rho
dtheta = lambda rho, theta, m: 0
map_circles(0.17, 0, drho, dtheta, ax=ax1, ls="--", lw=0.85)

# 2nd Coma
A = 10 * (1 + 4 * 0.61**2 + 0.61**4)
B = 12 * (1 + 4 * 0.61**2 + 4 * 0.61**4 + 0.61**6)
C = 3 * (1 + 4 * 0.61**2 + 10 * 0.61**4 + 4 * 0.61**6 + 0.61**8)
drho = lambda rho, theta, m: -(5 * A * rho**4 - 3 * B * rho**2 + C) * np.sin(theta)
dtheta = lambda rho, theta, m: -(A * rho**4 - B * rho**2 + C) * np.cos(theta)
map_circles(0.003, 0, drho, dtheta, ax=ax2, ls="--", lw=0.85, c="C1")

plt.show()